# Загрузка данных и фильтрация


In [16]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import time

# Загрузка датасета с ирисами
iris = load_iris()

# Преобразование данных в pandas DataFrame
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

# Фильтрация данных оставляет только Versicolor (класс 1) и Virginica (класс 2)
filtered_iris_df = iris_df[iris_df['target'].isin([1, 2])]

# Вывод первых нескольких строк фильтрованного датасета
print(filtered_iris_df.head())

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
50                7.0               3.2                4.7               1.4   
51                6.4               3.2                4.5               1.5   
52                6.9               3.1                4.9               1.5   
53                5.5               2.3                4.0               1.3   
54                6.5               2.8                4.6               1.5   

    target  
50       1  
51       1  
52       1  
53       1  
54       1  


In [2]:
X = filtered_iris_df.drop('target', axis=1)
y = filtered_iris_df['target']

In [3]:
y -= 1
y


50     0
51     0
52     0
53     0
54     0
      ..
145    1
146    1
147    1
148    1
149    1
Name: target, Length: 100, dtype: int64

# Метод градиентного спуска

In [4]:
def logistic_regression_gradient(X, y, num_iterations, learning_rate):
    # Инициализация весов
    num_features = X.shape[1]
    weights = np.zeros(num_features)
    bias = 0

    # Процесс обновления весов
    for i in range(num_iterations):
        # Линейная комбинация входных признаков и весов
        linear_combination = np.dot(X, weights) + bias

        # Применение функции сигмоиды
        predicted_probabilities = 1 / (1 + np.exp(-linear_combination))

        # Вычисление градиента для весов и смещения
        gradient_weights = np.dot(X.T, (predicted_probabilities - y)) / len(X)
        gradient_bias = np.mean(predicted_probabilities - y)

        # Обновление весов и смещения
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_bias

    return weights, bias

In [5]:
def predict(X, weights, bias):
    # Линейная комбинация входных признаков и весов
    linear_combination = np.dot(X, weights) + bias

    # Применение функции сигмоиды
    predicted_probabilities = 1 / (1 + np.exp(-linear_combination))

    # Округление вероятностей до бинарных предсказаний
    predicted_labels = np.round(predicted_probabilities)

    return predicted_labels

def accuracy(X, y, weights, bias):
    # Получение предсказаний
    predicted_labels = predict(X, weights, bias)

    # Вычисление точности предсказаний
    accuracy = np.mean(predicted_labels == y)

    return accuracy

In [17]:
start_time = time.time()
learning_rate = 0.01
num_iterations = 1000
weights, bias = logistic_regression_gradient(X, y, num_iterations, learning_rate)
end_time = time.time()
total_time1 = end_time - start_time

In [7]:
acc1 = accuracy(X, y, weights, bias)
print("Точность предсказаний logistic_regression_gradient:", acc1)

Точность предсказаний logistic_regression_gradient: 0.96


# Метод скользящего среднего

In [18]:
import numpy as np
import pandas as pd

def logistic_regression_smoothing(X, y, num_iterations, learning_rate, smoothing_factor):
    # Инициализация весов
    num_features = X.shape[1]
    weights = np.zeros(num_features)
    bias = 0

    # Инициализация переменных для скользящего среднего
    weights_avg = np.zeros(num_features)
    bias_avg = 0
    t = 1  # Шаг скользящего среднего

    # Процесс обновления весов
    for i in range(num_iterations):
        # Линейная комбинация входных признаков и весов
        linear_combination = np.dot(X, weights) + bias

        # Применение функции сигмоиды
        predicted_probabilities = 1 / (1 + np.exp(-linear_combination))

        # Вычисление градиента для весов и смещения
        gradient_weights = np.dot(X.T, (predicted_probabilities - y)) / len(X)
        gradient_bias = np.mean(predicted_probabilities - y)

        # Обновление весов и смещения
        weights -= learning_rate * gradient_weights

        # Обновление переменных скользящего среднего
        weights_avg = (smoothing_factor * weights_avg + (1 - smoothing_factor) * weights) / t
        bias_avg = (smoothing_factor * bias_avg + (1 - smoothing_factor) * bias) / t

        # Обновление смещения
        bias -= learning_rate * gradient_bias

        # Увеличение шага скользящего среднего
        t += 1

    return weights_avg, bias_avg

In [19]:
start_time = time.time()
# Обучение модели
learning_rate = 0.01
num_iterations = 1000
smoothing_factor = 0.9
weights, bias = logistic_regression_smoothing(X, y, num_iterations, learning_rate, smoothing_factor)
end_time = time.time()
total_time2 = end_time - start_time

In [20]:
# Оценка точности предсказаний
acc2 = accuracy(X, y, weights, bias)
print("Точность предсказаний logistic_regression_smoothing:", acc2)

Точность предсказаний logistic_regression_smoothing: 0.96


# Метод адаптивной оценки моментов

In [21]:
import numpy as np
import pandas as pd

def logistic_regression_nadam(X, y, num_iterations, learning_rate, beta1, beta2, epsilon):
    # Инициализация весов
    num_features = X.shape[1]
    weights = np.zeros(num_features)
    bias = 0

    # Инициализация моментов первого и второго порядка
    m_weights = np.zeros(num_features)
    m_bias = 0
    v_weights = np.zeros(num_features)
    v_bias = 0

    # Процесс обновления весов
    for i in range(num_iterations):
        # Вычисление градиента для весов и смещения
        gradients_weights, gradient_bias = compute_gradients(X, y, weights, bias)

        # Обновление моментов первого порядка
        m_weights = beta1 * m_weights + (1 - beta1) * gradients_weights
        m_bias = beta1 * m_bias + (1 - beta1) * gradient_bias

        # Обновление моментов второго порядка
        v_weights = beta2 * v_weights + (1 - beta2) * np.square(gradients_weights)
        v_bias = beta2 * v_bias + (1 - beta2) * np.square(gradient_bias)

        # Коррекция моментов первого и второго порядка
        m_weights_hat = m_weights / (1 - np.power(beta1, i+1))
        m_bias_hat = m_bias / (1 - np.power(beta1, i+1))
        v_weights_hat = v_weights / (1 - np.power(beta2, i+1))
        v_bias_hat = v_bias / (1 - np.power(beta2, i+1))

        # Обновление весов и смещения с использованием Nadam
        weights -= learning_rate * (m_weights_hat / (np.sqrt(v_weights_hat) + epsilon))
        bias -= learning_rate * (m_bias_hat / (np.sqrt(v_bias_hat) + epsilon))

    return weights, bias

def compute_gradients(X, y, weights, bias):
    # Линейная комбинация входных признаков и весов
    linear_combination = np.dot(X, weights) + bias

    # Применение функции сигмоиды
    predicted_probabilities = 1 / (1 + np.exp(-linear_combination))

    # Вычисление градиента для весов и смещения
    gradients_weights = np.dot(X.T, (predicted_probabilities - y)) / len(X)
    gradient_bias = np.mean(predicted_probabilities - y)

    return gradients_weights, gradient_bias


In [22]:
tart_time = time.time()
# Обучение модели
learning_rate = 0.01
num_iterations = 1000
beta1 = 0.9
beta2 = 0.999
epsilon = 1e-8
weights, bias = logistic_regression_nadam(X, y, num_iterations, learning_rate, beta1, beta2, epsilon)

end_time = time.time()
total_time3 = end_time - start_time

In [23]:
# Оценка точности предсказаний
acc3 = accuracy(X, y, weights, bias)
print("Точность предсказаний logistic_regression_nadam:", acc3)

Точность предсказаний logistic_regression_nadam: 0.97


# Сравнение результатов

In [24]:
result = pd.DataFrame()
result["logistic_regression_gradient"] = [acc1,  total_time1]
result["logistic_regression_smoothing"] = [acc2,  total_time2]
result["logistic_regression_nadam"] = [acc3,  total_time3]
result.index = ['Accuracy', 'Time']

In [25]:
result

,logistic_regression_gradient,logistic_regression_smoothing,logistic_regression_nadam
Accuracy,0.960000,0.960000,0.970000
Time,0.396735,0.465906,1.752445


Наибольшей точностью в нашем случае обладает метод адаптивной оценки моментов, но в то же время его реализация занимает более чем в 200 раз больше времени по сравнению с остпальными методами. Поэтому считаю, что с точки зрения экономии ресурсов разница в точности в 1% совсем не критична, и можно пользоваться методом градиентного спуска или скользящего среднего. эти методы также показывают высокую точность - 96%